In [1]:
import json
import random, string
import pandas as pd
import numpy as np
from collections import defaultdict
import requests


# Functions

## Create Flow Steps

In [2]:
def createflowsteps (title, actionId, intentId, stepId):#
    return [{'meta': {'audioHashes': []},
   'type': 'INTENT',
   'title': title,
   'contexts': [],
   'actions': [{'actionId': actionId}],
   'children': [],
   'stepId': stepId,
   'intent': {'intentId': intentId}}]
# createflowsteps ('Intent 1','a','i','s') 

## Create Flows

In [3]:
def createflow (title, actionId, flowId, intentId, stepId):
    return {
        'flowId': '4cc34237-cbd9-4e08-8a4e-d1104c34ea5b',
        'disabled': False,
        'group': 'FAQ',
        'metadata': [],
        'steps': createflowsteps (title, actionId, intentId, stepId),
        'title': title}
# createflow ('title', "actionId", "flowId", "intentId", "stepId")

## Create Intents

In [4]:
def createintent (title, brainId, intentId):
    return {
        'brainId': brainId,
        'intentId': intentId,
        'accuracy': 15.38,
        'disableTraining': False,
        'examples': [
            {'entities': [], 'query': title},
            ],
         'title': title}
# createintent ('title', "brains", "intents")

## Create Actions

In [5]:
def createaction (actionId, brainId, answer):
    return {
        'actionId': actionId,
        'brainId': brainId,
        'payload': {'texts': [answer], 'quickReplies': [], 'tags': []},
        'type': 'TEXT'}
# createintent ('title', "brains", "intents")

## Text Generation Service

In [6]:
# TextGen API
def textgen(sentence, lang):
    
    url = "https://textgenapp-259408.appspot.com/textgen"

    payload = {
        'input sentences': [sentence],
        'lang': lang
    }
    headers = {
        'Content-Type': "application/json",
        'User-Agent': "PostmanRuntime/7.19.0",
        'Accept': "*/*",
        'Cache-Control': "no-cache",
        'Postman-Token': "e6565f73-3b8d-4dff-82c6-8b5ef3c969ec,6118f8fa-8824-419b-8558-5e84893cc8c9",
        'Host': "textgenapp-259408.appspot.com",
        'Accept-Encoding': "gzip, deflate",
        'Content-Length': "89",
        'Connection': "keep-alive",
        'cache-control': "no-cache"
        }
    
    example = str(json.dumps(payload))
    # print(example)
    response = requests.request("POST", url, data=example, headers=headers)
    return(response)
    

In [7]:
# Get Trainings Data from API in list
def get_td (sentence, lang):
    response = textgen(sentence,lang)
    json_data = json.loads(response.text)
    td = [item[0] for item in json_data]
    return td

In [8]:
# create training examples
def create_td (list_sentences):
    td = []
    for example in list_sentences:  
         td.append({'entities': [], 'query': example})
    return td

## Key Generator

In [9]:
def keygen():
    k = ''.join(random.choice(string.ascii_lowercase + string.digits) for _ in range(16))
    return k

# Get FAQ's from Spreadsheet


In [10]:
def create_flows_intents_actions (project, list_of_questions, list_of_answers):
    keys = ["actionId","intentId", "flowId", "stepId"]
    brainId = keygen()
    
    flows = []
    intents = []
    actions = []
    
    for i in range(len(questions)):
        title = questions[i]
        answer = answers[i]
        dkeys = {}
        for key in keys:
            dkeys[key]=keygen()
        #print(dkeys)
        flows.append (createflow (title, dkeys["actionId"] ,dkeys["flowId"], dkeys["intentId"], dkeys["stepId"]))
        intents.append (createintent (title, brainId, dkeys["intentId"]))
        actions.append (createaction (dkeys["actionId"], brainId, answer) )
        #print()
        
    project['brains'][0]['flows'] = flows
    project['brains'][0]['intents'] = intents
    project['brains'][0]['actions'] = actions
    return project




keys = ["actionId","intentId", "flowId", "stepId"]
brainId = keygen()

flows = []
intents = []
actions = []
for i in range(len(questions)):
    title = questions[i]
    answer = answers[i]
    dkeys = {}
    for key in keys:
        dkeys[key]=keygen()
    #print(dkeys)
    flows.append (createflow (title, dkeys["actionId"] ,dkeys["flowId"], dkeys["intentId"], dkeys["stepId"]))
    intents.append (createintent (title, brainId, dkeys["intentId"]))
    actions.append (createaction (dkeys["actionId"], brainId, answer) )
    #print()

# Add training examples from TextGen


In [11]:
def add_textgen (project):
    intents = project['brains'][0]['intents']
    for i in range(len(intents)):
        examples = intents[i]['examples']
        in_tg = examples[0]['query']
        listtd = get_td (in_tg, "nl")
        listtd.append(in_tg)
        examples = create_td (listtd)
        project['brains'][0]['intents'][i]['examples']=examples
    return project


In [12]:
#with open('uploadthisfiletoflow.json') as json_file:


In [13]:
#with open('uploadwithexamples.json', 'w') as f:
#    json.dump(tg, f)

# Merge with base

In [14]:


def merge_flows (prjct_base,prjct_head):
    for flow in prjct_head['brains'][0]['flows']:
        prjct_base['brains'][0]['flows'].append(flow)
    return prjct_base

def merge_intents (prjct_base,prjct_head):
    brainId = prjct_base['brains'][0]['brain']['brainId']
    for intent in prjct_head['brains'][0]['intents']:
        prjct_base['brains'][0]['intents'].append(intent)
    return prjct_base

def merge_actions (prjct_base,prjct_head):
    brainId = prjct_base['brains'][0]['brain']['brainId']
    for action in prjct_head['brains'][0]['actions']:
        prjct_base['brains'][0]['actions'].append(action)
    return prjct_base

# Replace with organzation info & carousel image

In [15]:
def find_flow (project, flowname):
    for flow in merged['brains'][0]['flows']:
        if flow['title']==flowname:
            return(flow)


In [16]:
def find_action (project,actionId):
    for action in merged['brains'][0]['actions']:
        if action['actionId']==actionId:
            return(action)
        
def find_integration (project,integrationId):
    for i in range(len(project['integrations'])):
        if project['integrations']['integrationId']==integrationId:
            print("integration index : ",i)
            return project['integrations'][i]
        


In [17]:
# fill opening with organization name and chatbot name
def create_opening (project, organizationName, chatbotName):
    
    opening = {'actionId': '3d721dcc-0978-4604-824b-d6c904fc94a2',
     'brainId': '2233f6d1-2817-4ff2-bca4-e095918e66c8',
     'payload': {'texts': ['Welkom bij {}! Ik ben {}, jouw virtuele assistent.'.format((organizationName),chatbotName)],
      'quickReplies': [],
      'tags': []},
     'type': 'TEXT'}
    
    project['brains'][0]['actions'][13]=opening
    return project



In [18]:

def create_card (topic, list_of_questions, imgurl):
    return {'title': 'Swipe en select below <👇>',
      'subtitle': topic,
      'buttons': create_button (list_of_questions),
      'media': create_media ('https://source.unsplash.com/featured/?'+str(topic))}

In [19]:
def create_button (list_of_questions):
    buttons = []
    for question in list_of_questions:
        buttons.append({'label': question, 'type': 'postback', 'value': question})
        if len(buttons)==3:
            continue
    return buttons

#images only
def create_media (imgurl):
    return {'type': 'image','url': imgurl}



In [20]:
def create_carousel (project, list_unique_topics):
    opening_carousel = {'actionId': 'fa67d2e1-6b4e-42e1-9107-548a47282970',
     'brainId': '2233f6d1-2817-4ff2-bca4-e095918e66c8',
     'payload': {'fallback': '',
      'response': {'type': 'carousel',
       'payload': {'cards': []
                  }
                  },
      'tags': []},
     'type': 'CAROUSEL'}
    
    for topic in unique_topics:
        q = list(df.loc[df['Topic'] == topic, 'Questions'])
        card = create_card (topic, q, "imgurl")
        opening_carousel['payload']['response']['payload']['cards'].append(card)
    
    project['brains'][0]['actions'][4] = opening_carousel
    return project
    


# Replace Handoff email adress

In [21]:
def replace_handoff_email (project, supportemail):
    integration = project['integrations'][3]
    CC = integration['cloudCode']
    CC = CC.replace("Support Email",(supportemail))
    integration['cloudCoude']= CC
    project['integrations'][3]['cloudCode'] = CC
    return project

In [24]:
# get FAQs

# Open base
with open('export.json') as json_file:
    data = json.load(json_file)

# Work with XSLX and define variables
xls = pd.ExcelFile('FAQ.xlsx')
df1 = pd.read_excel(xls, "FAQ's")
df = pd.DataFrame()
df['Questions']=list(df1['Question'])
df['Answers']=list(df1['Answer'])
df['Topic']=list(df1['Topic'])
n_items = max(df.count())
qa = df[:n_items]
# get Organization Info
df2 = pd.read_excel(xls, "Introduction")
df2.columns = ['row', 'faq2bot instruction', 'none', 'label', 'value']
orgdict = df2.set_index('label')['value'].to_dict()
questions = list(qa['Questions'])
answers = list(qa['Answers'])

#create from excel
data = create_flows_intents_actions (data, questions, answers)

tg = add_textgen (data)
with open('general nl1.json') as json_file:
    a = json.load(json_file)
b = tg #change to tg / data

new = merge_flows (a,b)
new = merge_intents (new,b)
merged = merge_actions (new,b)

merged = create_opening (merged, orgdict['Organization'], orgdict['Chatbot Name'])
unique_topics = sorted(df.Topic.unique())
merged = create_carousel (merged, unique_topics)
merged = replace_handoff_email (merged, orgdict['Support Email'])

In [25]:
with open('merged_projects.json', 'w') as f:
    json.dump(merged, f)

In [28]:
find_flow (merged, "02 Opening")

{'group': '00 General',
 'title': '02 Opening',
 'disabled': False,
 'flowId': 'b71a9c91-d13a-4ce6-a59a-e09a36860645',
 'metadata': [],
 'steps': [{'meta': {'audioHashes': []},
   'type': 'EVENT',
   'title': 'Opening',
   'contexts': [],
   'actions': [{'actionId': '3d721dcc-0978-4604-824b-d6c904fc94a2'},
    {'actionId': '43f36c26-3e9d-412d-8eca-1eb8507a85d0'},
    {'actionId': '1f876bb9-911b-4139-82a2-8468a464ac1a'}],
   'stepId': '36b03cdb-0afa-4ea7-b3c0-63bf9cd1d4fc',
   'intent': None}],
 'isChannelSpecific': False}

In [ ]:
# '43f36c26-3e9d-412d-8eca-1eb8507a85d0'

In [30]:
find_action (merged,'43f36c26-3e9d-412d-8eca-1eb8507a85d0')


{'actionId': '43f36c26-3e9d-412d-8eca-1eb8507a85d0',
 'brainId': '2233f6d1-2817-4ff2-bca4-e095918e66c8',
 'payload': {'texts': ['Stel je vraag direct of selecteer het topic van jouw vraag zodat ik je verder kan helpen. ^Chatbot Name 🤖'],
  'quickReplies': [],
  'delay': 2000,
  'tags': []},
 'type': 'TEXT'}

In [ ]:
{'actionId': '43f36c26-3e9d-412d-8eca-1eb8507a85d0',
 'brainId': '2233f6d1-2817-4ff2-bca4-e095918e66c8',
 'payload': {'texts': ['Stel je vraag direct of selecteer het topic van jouw vraag zodat ik je verder kan helpen. ^Chatbot Name 🤖'],
  'quickReplies': [],
  'delay': 2000,
  'tags': []},
 'type': 'TEXT'}

In [ ]:
def replace_text_in_reply (project, actionId, replace, replace_with):
     = project['integrations'][3]
    CC = integration['cloudCode']
    CC = CC.replace("Support Email",(supportemail))
    integration['cloudCoude']= CC
    project['integrations'][3]['cloudCode'] = CC
    return project

In [41]:
def replace_text_in_reply (project, actionId, replace, replace_with):
    actions = project['brains'][0]['actions']
    for i in range(len(actions)):
        if actions[i]['actionId']==actionId:
            actions[i]['payload']['texts'][0] = actions[i]['payload']['texts'][0].replace(replace, replace_with)
    return project
replace_text_in_reply (merged, '43f36c26-3e9d-412d-8eca-1eb8507a85d0', "Chatbot Name", orgdict['Chatbot Name'])

index of intent: 14
{'actionId': '43f36c26-3e9d-412d-8eca-1eb8507a85d0', 'brainId': '2233f6d1-2817-4ff2-bca4-e095918e66c8', 'payload': {'texts': ['Stel je vraag direct of selecteer het topic van jouw vraag zodat ik je verder kan helpen. ^Pablo 🤖'], 'quickReplies': [], 'delay': 2000, 'tags': []}, 'type': 'TEXT'}
{'actionId': '43f36c26-3e9d-412d-8eca-1eb8507a85d0', 'brainId': '2233f6d1-2817-4ff2-bca4-e095918e66c8', 'payload': {'texts': ['Stel je vraag direct of selecteer het topic van jouw vraag zodat ik je verder kan helpen. ^Pablo 🤖'], 'quickReplies': [], 'delay': 2000, 'tags': []}, 'type': 'TEXT'}


{'brains': [{'brain': {'threshold': 70,
    'pauseTimeout': 15,
    'brainId': '2233f6d1-2817-4ff2-bca4-e095918e66c8',
    'brainName': 'flowai',
    'description': 'Flow.ai',
    'isDefault': True,
    'language': 'nl',
    'title': 'General'},
   'flows': [{'group': '00 General',
     'title': '01 Groet',
     'disabled': False,
     'flowId': '2e2bc877-dc84-45fa-9908-6308decd2ffe',
     'metadata': [],
     'steps': [{'type': 'INTENT',
       'title': 'Groet',
       'contexts': [],
       'actions': [{'actionId': '1f699609-493e-4c37-afe6-0c1ab041705a'}],
       'stepId': '609ce795-b703-4a6e-8d10-05f6444c379f',
       'intent': {'intentId': 'cec3b35c-d4bd-46a4-b6af-95a351965352'}}],
     'isChannelSpecific': False},
    {'group': '00 General',
     'title': '02 Opening',
     'disabled': False,
     'flowId': 'b71a9c91-d13a-4ce6-a59a-e09a36860645',
     'metadata': [],
     'steps': [{'meta': {'audioHashes': []},
       'type': 'EVENT',
       'title': 'Opening',
       'contexts': 